In [1]:
from Preprocessing import Preprocessing
from utils import Behaviour, Baseline
import Experiments
import matplotlib
import matplotlib.pyplot as plt
from multiprocessing import Pool
import math
matplotlib.use('Agg') # Prevent drawings in this notebook

In [ ]:
bids_root = "../data/"
preprocessing = Preprocessing(bids_root, "cache")
# create cache files
preprocessing.prepare_channels()

Reading /home/yannick/Documents/MasterInformatik/EEG/test/EEGAnalysis/Clean/../data/sub-001/eeg/sub-001_task-ContinuousVideoGamePlay_run-02_eeg.fdt
Reading events from ../data/sub-001/eeg/sub-001_task-ContinuousVideoGamePlay_run-02_events.tsv.
The event "STATUS" refers to multiple event values. Creating hierarchical event names.
    Renaming event: STATUS -> STATUS/boundary
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATUS/S 98
    Renaming event: STATUS -> STATU

/home/yannick/Documents/MasterInformatik/EEG/test/EEGAnalysis/Clean/Preprocessing.py:78: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = read_raw_bids(bids_path)
/home/yannick/Documents/MasterInformatik/EEG/test/EEGAnalysis/Clean/Preprocessing.py:78: RuntimeWarning: No BIDS -> MNE mapping found for channel type "n/a". Type of channel "Fp1" will be set to "misc".
  raw = read_raw_bids(bids_path)
/home/yannick/Documents/MasterInformatik/EEG/test/EEGAnalysis/Clean/Preprocessing.py:78: RuntimeWarning: No BIDS -> MNE mapping found for channel type "n/a". Type of channel "Fz" will be set to "misc".
  raw = read_raw_bids(bids_path)
/home/yannick/Documents/MasterInformatik/EEG/test/EEGAnalysis/Clean/Preprocessing.py:78: RuntimeWarning: No BIDS -> MNE mapping found for channel type "n/a". Type of channel "F3" will be set to "misc".
  raw = read_raw_bids(bids_path)
/home/yannick/Documents/Mas

Reading 0 ... 1398674  =      0.000 ...  2797.348 secs...
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1651 samples (3.302 s)



/home/yannick/Documents/MasterInformatik/EEG/test/EEGAnalysis/Clean/Preprocessing.py:104: RuntimeWarning: The unit for channel(s) AF3, AF4, AF7, AF8, AFz, C1, C2, C3, C4, C5, C6, CP1, CP2, CP3, CP4, CP5, CP6, Cz, F1, F2, F3, F4, F5, F6, F7, F8, FC1, FC2, FC3, FC4, FC5, FC6, FCz, FT10, FT7, FT8, FT9, Fp1, Fp2, Fz, HEOG, O1, O2, Oz, P1, P2, P3, P4, P5, P6, P7, P8, PO3, PO4, PO7, PO8, POz, Pz, T7, T8, TP10, TP7, TP8, TP9, VEOG has changed from NA to V.
  raw.set_channel_types(dict(zip(list_name, list_type)))
/home/yannick/Documents/MasterInformatik/EEG/test/EEGAnalysis/Clean/Preprocessing.py:117: RuntimeWarning: Not setting positions of 2 eog channels found in montage:
['VEOG', 'HEOG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw.set_montage(montage)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s


Fitting ICA to data using 65 channels (please be patient, this may take a while)
Selecting by number: 30 components


In [ ]:
combinations = [(i, behaviour) for i in range(65) for behaviour in Behaviour]
combinations.sort(key=lambda x: x[0])

process_count = 16
inputs = []
step_size = math.ceil(len(combinations)/process_count)
# split combinations to create inputs for processes
for i in range(process_count):
    inputs.append((combinations[step_size*i:step_size*(i+1)], preprocessing))

with Pool(process_count) as p:
    result = p.starmap(Experiments.run, inputs)

  3%|▎         | 1/29 [00:24<11:29, 24.61s/it]

In [ ]:
# join results from multiple processes
peak_to_troughs = {}
for behavior in Behaviour:
    peak_to_troughs[behavior] = {}
for i in range(len(result)):
    for behavior in result[i]:
        peak_to_troughs[behavior].update(result[i][behavior])

In [ ]:
def plot_peak_to_trough_differences(behaviour, baseline, mode):
    data = []
    for electrode_index, peak_to_trough_value in peak_to_troughs[behaviour].items():
        data.append((electrode_index, peak_to_trough_value[mode][baseline]))
    title = f"Lower Peak-to-Trough Differences for {behaviour.value} ({'Raw' if mode == 0 else 'ICA'}, {baseline.value})"
    sorted_values = sorted(data, key=lambda x:x[1])[:20]
    labels, values = zip(*sorted_values)
    plt.figure()
    plt.barh(range(len(values)), values, color='skyblue')
    plt.xlabel("Peak-to-Trough Difference")
    plt.ylabel("Electrode")
    plt.yticks(range(len(labels)), labels)
    plt.title(title)
    plt.savefig(f"../Results/{title}.png")
    plt.close()    

In [ ]:
for behavior in peak_to_troughs:
    for baseline in Baseline:
        plot_peak_to_trough_differences(behavior, baseline, 0)
        plot_peak_to_trough_differences(behavior, baseline, 1)